# Training a MLP using backpropagation and forward differentiation

In **part 1** we train using backpropagation, while in **part 2** we train using forward ad.

In [22]:
import jax.numpy as jnp
import jax
from net import MLP
from functional import relu, log_softmax
from jax import grad, jit

## Part 1
We first define useful parameters that we are going to use later. Note that our model is a MLP imported from `net.py`

In [28]:
# hyperparameters
layer_sizes = [784, 512, 512, 10]
step_size = 0.01
num_epochs = 8
batch_size = 128
n_targets = 10
key = jax.random.PRNGKey(10)

# to create a model is as simple as that
model = MLP(layer_sizes, relu, key)
params = model.params # we need to extract parameters

Helper functions, trainloader and other stuff...

In [24]:
# some helper functions
def one_hot(x, k, dtype=jnp.float32):
  """Create a one-hot encoding of x of size k."""
  return jnp.array(x[:, None] == jnp.arange(k), dtype)
  
def accuracy(params, images, targets):
  target_class = jnp.argmax(targets, axis=1)
  predicted_class = jnp.argmax(model(params, images), axis=1)
  return jnp.mean(predicted_class == target_class)

def loss(params, images, targets):
  preds = model(params, images)
  preds = log_softmax(preds)
  return -jnp.mean(preds * targets)

@jit
def update(params, x, y):
  grads = grad(loss)(params, x, y)
  return [(w - step_size * dw, b - step_size * db)
          for (w, b), (dw, db) in zip(params, grads)]


import numpy as np
from torch.utils import data
from torchvision.datasets import MNIST

def numpy_collate(batch):
  if isinstance(batch[0], np.ndarray):
    return np.stack(batch)
  elif isinstance(batch[0], (tuple,list)):
    transposed = zip(*batch)
    return [numpy_collate(samples) for samples in transposed]
  else:
    return np.array(batch)

class NumpyLoader(data.DataLoader):
  def __init__(self, dataset, batch_size=1,
                shuffle=False, sampler=None,
                batch_sampler=None, num_workers=0,
                pin_memory=False, drop_last=False,
                timeout=0, worker_init_fn=None):
    super(self.__class__, self).__init__(dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        sampler=sampler,
        batch_sampler=batch_sampler,
        num_workers=num_workers,
        collate_fn=numpy_collate,
        pin_memory=pin_memory,
        drop_last=drop_last,
        timeout=timeout,
        worker_init_fn=worker_init_fn)

class FlattenAndCast(object):
  def __call__(self, pic):
    return np.ravel(np.array(pic, dtype=jnp.float32))

# Define our dataset, using torch datasets
mnist_dataset = MNIST('/tmp/mnist/', download=True, transform=FlattenAndCast())
training_generator = NumpyLoader(mnist_dataset, batch_size=batch_size, num_workers=0)

# Get the full train dataset (for checking accuracy while training)
train_images = np.array(mnist_dataset.train_data).reshape(len(mnist_dataset.train_data), -1)
train_labels = one_hot(np.array(mnist_dataset.train_labels), n_targets)

# Get full test dataset
mnist_dataset_test = MNIST('/tmp/mnist/', download=True, train=False)
test_images = jnp.array(mnist_dataset_test.test_data.numpy().reshape(len(mnist_dataset_test.test_data), -1), dtype=jnp.float32)
test_labels = one_hot(np.array(mnist_dataset_test.test_labels), n_targets)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Here we train using backpropagation

In [9]:
import time

for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in training_generator:
    y = one_hot(y, n_targets)
    params = update(params, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

Epoch 0 in 9.15 sec
Training set accuracy 0.9136999845504761
Test set accuracy 0.9180999994277954
Epoch 1 in 8.54 sec
Training set accuracy 0.9374666810035706
Test set accuracy 0.9391999840736389
Epoch 2 in 8.06 sec
Training set accuracy 0.9483166933059692
Test set accuracy 0.9483999609947205
Epoch 3 in 8.11 sec
Training set accuracy 0.9550333619117737
Test set accuracy 0.9530999660491943
Epoch 4 in 8.00 sec
Training set accuracy 0.9599666595458984
Test set accuracy 0.9565999507904053
Epoch 5 in 8.00 sec
Training set accuracy 0.9637500047683716
Test set accuracy 0.9598000049591064
Epoch 6 in 7.99 sec
Training set accuracy 0.9667500257492065
Test set accuracy 0.9623000025749207
Epoch 7 in 8.25 sec
Training set accuracy 0.9694833159446716
Test set accuracy 0.9633999466896057


## Part 2

Now lets's train using forward AD. Note that we need to change a bit the function `update`

In [26]:
from jax import jvp

def get_vector(key, params):
  v_shaped = []
  for w,b in params:
    key, subkey = jax.random.split(key)
    v_w = jax.random.normal(key, shape=w.shape)
    v_b = jax.random.normal(subkey, shape=b.shape)
    v_shaped.append((v_w, v_b))
  return v_shaped

@jit
def update(params, key, x, y):
  v = get_vector(key, params)
  _, dd = jvp(lambda params: loss(params, x, y), (params,), (v,))
  step = step_size * dd
  return [(w - step * dw, b - step * db)
          for (w, b), (dw, db) in zip(params, v)]

Let's train again

In [29]:
step_size = 2e-5

for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in training_generator:
    key, _ = jax.random.split(key)
    y = one_hot(y, n_targets)
    params = update(params, key, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))


Epoch 0 in 13.03 sec
Training set accuracy 0.3375166654586792
Test set accuracy 0.3328000009059906
Epoch 1 in 12.73 sec
Training set accuracy 0.5393666625022888
Test set accuracy 0.5331000089645386
Epoch 2 in 12.53 sec
Training set accuracy 0.6308666467666626
Test set accuracy 0.6401000022888184
Epoch 3 in 15.68 sec
Training set accuracy 0.6854166984558105
Test set accuracy 0.6976999640464783
Epoch 4 in 15.73 sec
Training set accuracy 0.7157333493232727
Test set accuracy 0.7224999666213989
Epoch 5 in 12.70 sec
Training set accuracy 0.7372666597366333
Test set accuracy 0.7444999814033508
Epoch 6 in 13.06 sec
Training set accuracy 0.7376000285148621
Test set accuracy 0.7483999729156494
Epoch 7 in 12.83 sec
Training set accuracy 0.7309666872024536
Test set accuracy 0.7409999966621399
